In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from glob import glob

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
featuresA = ["has_name", "has_image", "has_location",
             "has_biography", "tweet_count",
             "followers_count", "friends_count", "ratio_followers_friends",
             "has_url", "bot_in_biography",
             "is_listed"]

In [4]:
cresci2015files = glob("datas/cresci-2015/*/users.csv")
tmp = []
for f in cresci2015files:
    df = pd.read_csv(f, index_col=None, header=0)
    tmp.append(df)
cresci2015 = pd.concat(tmp)
cresci2015 = cresci2015.set_index("id")

In [5]:
cresci2015["tweet_count"] = cresci2015.statuses_count
cresci2015 = cresci2015.fillna(0)
replace_from = ["name", "location", "default_profile_image", "description", "url", "listed_count"]
replace_to = ["has_name", "has_location", "has_image", "has_biography", "has_url", "is_listed"]
cresci2015[replace_to] = cresci2015[replace_from].astype(bool)
cresci2015["ratio_followers_friends"] = cresci2015.followers_count / cresci2015.friends_count
cresci2015["bot_in_biography"] = cresci2015.description.str.contains("\Wbot\W").fillna(False)

In [6]:
cresci2015

,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,url,lang,...,dataset,tweet_count,has_name,has_location,has_image,has_biography,has_url,is_listed,ratio_followers_friends,bot_in_biography
id,,,,,,,,,,,,,,,,,,,,,
1120176186,Madonna Carson,MadonnaCarson,22,11,257,0,0,Fri Jan 25 20:18:39 +0000 2013,0,en,...,FSF,22,True,True,False,True,False,False,0.042802,False
1124249136,Ardelle Mckee,ArdelleMckee,20,11,226,0,0,Sun Jan 27 07:18:34 +0000 2013,0,en,...,FSF,20,True,True,False,False,False,False,0.048673,False
1124264132,Isadora Wilson,IsadoraWilson,19,13,281,0,0,Sun Jan 27 07:19:28 +0000 2013,0,en,...,FSF,19,True,True,False,True,False,False,0.046263,False
1124293891,Genesis Merrill,GenesisMerrill,21,19,237,0,0,Sun Jan 27 07:33:40 +0000 2013,0,en,...,FSF,21,True,True,False,True,False,False,0.080169,False
1124295318,Kent Caldwell,KentCaldwell3,22,20,262,0,0,Sun Jan 27 07:43:52 +0000 2013,0,en,...,FSF,22,True,True,False,False,False,False,0.076336,False
1124301420,Drucilla Sellers,DrucillaSeller1,19,14,269,1,0,Sun Jan 27 07:47:15 +0000 2013,0,en,...,FSF,19,True,True,False,True,False,False,0.052045,False
1124314592,Helaine Taylor,HelaineTaylor,22,13,242,0,0,Sun Jan 27 07:46:58 +0000 2013,0,en,...,FSF,22,True,True,False,True,False,False,0.053719,False
1124348202,Senaida Hayes,SenaidaHayes,19,8,235,0,0,Sun Jan 27 08:11:56 +0000 2013,0,en,...,FSF,19,True,True,False,True,False,False,0.034043,False
1124360551,Mayra Glass,MayraGlass11,23,13,265,0,0,Sun Jan 27 08:09:15 +0000 2013,0,en,...,FSF,23,True,True,False,True,False,False,0.049057,False


In [7]:
X = cresci2015[featuresA].astype("float32").replace([np.inf, -np.inf], np.nan).fillna(0)
Y = (cresci2015.dataset == "TFP") | (cresci2015.dataset == "E13")
Y = Y.astype("int")

In [8]:
X

,has_name,has_image,has_location,has_biography,tweet_count,followers_count,friends_count,ratio_followers_friends,has_url,bot_in_biography,is_listed
id,,,,,,,,,,,
1120176186,1.0,0.0,1.0,1.0,22.0,11.0,257.0,0.042802,0.0,0.0,0.0
1124249136,1.0,0.0,1.0,0.0,20.0,11.0,226.0,0.048673,0.0,0.0,0.0
1124264132,1.0,0.0,1.0,1.0,19.0,13.0,281.0,0.046263,0.0,0.0,0.0
1124293891,1.0,0.0,1.0,1.0,21.0,19.0,237.0,0.080169,0.0,0.0,0.0
1124295318,1.0,0.0,1.0,0.0,22.0,20.0,262.0,0.076336,0.0,0.0,0.0
1124301420,1.0,0.0,1.0,1.0,19.0,14.0,269.0,0.052045,0.0,0.0,0.0
1124314592,1.0,0.0,1.0,1.0,22.0,13.0,242.0,0.053719,0.0,0.0,0.0
1124348202,1.0,0.0,1.0,1.0,19.0,8.0,235.0,0.034043,0.0,0.0,0.0
1124360551,1.0,0.0,1.0,1.0,23.0,13.0,265.0,0.049057,0.0,0.0,0.0


In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

In [11]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, Y_train)
clf.score(X_test, Y_test)

0.97371428571428575

In [13]:
for i,name in enumerate(featuresA):
    print("{} : {:.3}".format(name, clf.feature_importances_[i]))

has_name : 0.0
has_image : 0.0
has_location : 0.0477
has_biography : 0.0382
tweet_count : 0.164
followers_count : 0.177
friends_count : 0.0449
ratio_followers_friends : 0.503
has_url : 0.0259
bot_in_biography : 0.0
is_listed : 0.0
